# Funções

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from collections import defaultdict
import json
import warnings # retirar avisos
warnings.filterwarnings('ignore')

In [2]:
def load_data():
    """
    Função para ler e transformar os dados já presentes no diretório especificado
    """
    path = "../data/daily_load.csv"
    df_load = pd.read_csv(path, parse_dates = ["date"])
    df_load2 = df_load[df_load["id_reg"] == "S"]           # região sul
    df_load3 = df_load2[df_load2["date"] <= '2022-05-31']  # data de corte
    df_load4 = df_load3[["date", "load_mwmed"]].set_index("date")
    return df_load4

In [3]:
def train_test_split(data, n_test):
    """
    Função para partir or dados em treino e teste
    """
    if isinstance(data, pd.DataFrame):
        train, test = data.iloc[:-n_test, :], data.iloc[-n_test:, :]
    elif isinstance(data, np.ndarray):
        train, test = data[:-n_test, :], data[-n_test:, :]
    return train, test

In [4]:
def create_future(start, t, cal_vars = False):
    """ Função para criar DataFrame de datas (dias) seguintes a T, assim como as variáveis de calendário se cal_vars = True.
       start: T + 1
       t: períodos à frente """
    dates = pd.date_range(start, freq = 'd', periods = t)
    df = pd.DataFrame(dates, columns = ['t'])
    if cal_vars == True:
        df = create_features(df, 't')
    elif cal_vars == False:
        pass
    return df

In [5]:
def get_measures(forecast, test):
    """
    Função para obter medidas de acurária a partir dos dados de projeção e teste
    """
    forecast.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    if isinstance(forecast, pd.Series) and isinstance(test, pd.Series):
        errors = [(test.iloc[i] - forecast.iloc[i])**2 for i in range(len(test))]
    else:
        errors = [(test.iloc[i][0] - forecast.iloc[i])**2 for i in range(len(test))]
    mae = mean_absolute_error(test, forecast)
    mse = mean_squared_error(test, forecast)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(test, forecast)
    # smape
    a = np.reshape(test.values, (-1,))
    b = np.reshape(forecast.values, (-1,))
    smape = np.mean(100*2.0 * np.abs(a - b) / (np.abs(a) + np.abs(b))).item()
    # dicionário com as medidas de erro
    measures = { "erro": sum(errors),
                 "mae": mae,
                 "mse": mse,
                 "rmse": rmse,
                 "mape": mape,
                 "smape": smape
                }
    # arredondamento
    # for key, item in measures.items():
    #     measures[key] = round(measures[key], 2)
    return measures

# Prophet
Quick start: https://facebook.github.io/prophet/docs/quick_start.html <br>
Holidays and special events: https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html#modeling-holidays-and-special-events

In [6]:
import fbprophet # conda install -c conda-forge fbprophet

In [7]:
fbprophet.__version__ # verifica versão do Prophet

'0.7.1'

In [8]:
df = load_data().reset_index()
df.columns = ['ds','y']
df

,ds,y
0,2000-01-01,4800.650000
1,2000-01-02,4899.800000
2,2000-01-03,6261.554167
3,2000-01-04,6733.741667
4,2000-01-05,6961.170833
...,...,...
8183,2022-05-27,12520.803833
8184,2022-05-28,10525.490875
8185,2022-05-29,9074.211250
8186,2022-05-30,11648.709583


In [ ]:
# h = 15
# k_fold = 11

# df_cv = pd.DataFrame()
# for i in range(1, k_fold, 1):
#     desloc= h + k_fold 
#     df_ = df.iloc[:(-desloc+i+1), :]
#     print(df_.ds.min(), df_.ds.max())
#     m = fbprophet.Prophet(daily_seasonality=True)
#     model = m.fit(df_)
#     future = m.make_future_dataframe(periods = 15, freq = 'D')
#     prediction = m.predict(future)
#     pred_ = prediction[['ds','yhat']].iloc[-15:]
#     #test = df.iloc[(-desloc+i+1):]
#     pred_["ult_dt_train"] = df_.ds.max()
#     df_cv = pd.concat([df_cv, pred_], axis = 0)

# l = []
# for dv_date in df_cv_2.ult_dt_train.unique():
#     df_cv_2_ = df_cv_2[df_cv_2.ult_dt_train == dv_date]
#     acuracia = get_measures(df_cv_2_.yhat, df_cv_2_.y)
#     l.append(acuracia["mape"])
# print(l, sum(l)/len(l))

In [31]:
folds = 15 #partições
horz = n_test = 15 #horizonte de predição
rows = df.shape[0]
df_base = pd.DataFrame()
for fold in range(folds,0,-1):
    print(f"forecasting fold {fold}...")
    #slide = rows-(fold*horz)#-1
    slide = rows-((fold-1)*horz)
    df_cv = df.iloc[:slide]
    train, test = train_test_split(df, n_test)
    train.columns = ['ds', 'y']
    m = fbprophet.Prophet(daily_seasonality=True)
    model = m.fit(train)
    future = m.make_future_dataframe(periods = 15, freq = 'D')
    prediction = m.predict(future)
    pred_ = prediction[['ds','yhat']].iloc[-15:]
    pred_["ult_dt_train"] = df_cv.ds.max()
    df_base = pd.concat([df_base, pred_], axis = 0)
    out[f"cv_{fold}"]["pred"] = pred_["yhat"]
    out[f"cv_{fold}"]["test"] = test["y"]
d = dict(out)

forecasting fold 15...
forecasting fold 14...
forecasting fold 13...
forecasting fold 12...


KeyboardInterrupt: 

In [ ]:
mapes = []
for x in d:
    meas = get_measures(pd.Series(d[x]["pred"]),pd.Series(d[x]["test"]))
    print(meas)
    mapes.append(meas["mape"])

{'erro': 1681395.980425576, 'mae': 265.0477263651211, 'mse': 112093.06536170507, 'rmse': 334.80302472006593, 'mape': 0.024959958369317206, 'smape': 2.455641939496512}


In [ ]:
np.mean(mapes)

In [ ]:
with open('validation/prophet_cv.json', 'w') as f:
    json.dump(d, f)